In [38]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

In [61]:
def sacar_tuplas_moviedatabase(archivo):
    """
    Extrae las tuplas de datos de un archivo CSV.

    Parameters:
    archivo (str): La ruta del archivo CSV.

    Returns:
    list: Una lista de tuplas que contienen los datos extraídos del archivo CSV.
    """

    # Lee el archivo CSV y carga los datos en un DataFrame
    df_detalles = pd.read_csv(archivo)
    
    # Lista para almacenar las tuplas extraídas
    val = []
    
    # Lista para almacenar las filas duplicadas
    duplicadas = []


    # Recorre cada fila del DataFrame
    for pelicula in df_detalles.values:
    
        # Convierte la fila en una lista
        pelicula = list(pelicula)
        
        # Reemplaza los guiones simples '-' por None
        for i, elemento in enumerate(pelicula):
            if elemento == '-':
                pelicula[i] = None
     
        # Itera sobre cada elemento de la película
        for i, elemento in enumerate(pelicula):
            
            # Reemplaza los valores NaN con None
            if elemento is np.nan:
                pelicula[i] = None
            
                # Direccion y guion
                if isinstance(pelicula[2], list):
                    pelicula[2] = ', '.join(pelicula[2])

                if isinstance(pelicula[3], list):
                    pelicula[3] = ', '.join(pelicula[3])
                
                # Intenta convertir los elementos de tipo str a float o int si es posible
                if isinstance(elemento, str) and "." in elemento:

                    try:
                        pelicula[i] = float(elemento)
                    except:
                        pass
                
                if isinstance(elemento, str) and any(caracter.isdigit() for caracter in elemento):

                    try:
                        pelicula[i] = int(elemento)
                    except:
                        pass
                    
                if isinstance(elemento, str) and elemento.startswith('-') and elemento.endswith('-'):

                    try:
                        pelicula[i] = None
                    except:
                        pass
                    
        # Verifica si la película ya está en la lista de datos
        for dato in val:
            if pelicula[-1] in dato:
                duplicadas.append(pelicula)
            
        # Agrega la película a la lista de datos si no está duplicada
        if pelicula not in duplicadas:
            val.append(tuple(pelicula))
    
    # Retorna la lista de tuplas
    return val 

In [92]:
archivo = 'BBDD_Importaciones/Peliculas_detalles/detalles_pelis_2020_2023.csv'
prueba = sacar_tuplas_moviedatabase(archivo)


In [93]:
prueba

[(None,
  None,
  'Hector Echavarria',
  '[\'Brad Bode\', "Fuad C\'Amanero", \'Hector Echavarria\']',
  '1967 China, A young boy (Dax) is left to survive on his own in the middle of no were, after his missionary parents were murdered by rebels. He is found and Raised by a Shawling Temple Monk and becomes skilled in Martial arts. 25 Years later, in Los Angeles he meets a long time friend, that now works for the FBI, who asks for his help in solving a case revolving the tr... Leer todo',
  None,
  'Accion',
  'duel of legends',
  'tt0849437'),
 (None,
  '3.7',
  'Tito da Costa',
  "['Larry Leahy', 'Tito da Costa']",
  'A surf-skate action-thriller about five best friends who set out on a road trip that quickly turns into an epic battle for survival.',
  None,
  'Accion',
  'road to red',
  'tt0926132'),
 (None,
  '4.9',
  'Jeff Wadlow',
  "['Jeff Wadlow', 'Christopher Roach', 'Jillian Jacobs']",
  None,
  '109',
  'Accion',
  'fantasy island',
  'tt0983946'),
 ('60',
  '5.3',
  'Vincent 

In [94]:
# Establece la conexión con la el servidor MySQL y la BBDD

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto_cinemextract')



# Inserta los datos usando la llamada a la función en la tabla deseada

mycursor = cnx.cursor()
#query = "INSERT INTO detalles (PuntuacionRottenTomatoes, PuntuacionIMDB, Direccion, Guion, Argumento, Duracion, Genero, Titulo, IdPelicula) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
query = "INSERT INTO detalles (PuntuacionRottenTomatoes, PuntuacionIMDB, Direccion, Guion, Argumento, Duracion, Genero, Titulo, IdPelicula) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE PuntuacionRottenTomatoes=VALUES(PuntuacionRottenTomatoes), PuntuacionIMDB=VALUES(PuntuacionIMDB), Direccion=VALUES(Direccion), Guion=VALUES(Guion), Argumento=VALUES(Argumento), Duracion=VALUES(Duracion), Genero=VALUES(Genero), Titulo=VALUES(Titulo), IdPelicula=VALUES(IdPelicula)"

# Si el archivo de origen contiene índices, llamar a la funcion 'sacar_tuplas_i_moviedatabase'
val = sacar_tuplas_moviedatabase(archivo)

try: 
    mycursor.executemany(query, val)
    cnx.commit()
    print(mycursor.rowcount, "registro insertado.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)
    

4696 registro insertado.
